In [1]:
import pandas as pd
from keras.integration_test.preprocessing_test_utils import preprocessing
from keras.models import Model
from keras.layers import add, Input, Conv1D, Activation, Flatten, Dense


# 载入数据



def read_data(path):
    df = pd.read_csv(path)
    TIME_PERIODS = 160
    STEP_DISTANCE = 80
    features = ['DEPTH_MD', 'X_LOC', 'Y_LOC', 'Z_LOC',
                'CALI', 'RSHA', 'RMED', 'RDEP', 'RHOB', 'GR', 'NPHI', 'PEF', 'DTC',
                'SP', 'BS', 'ROP', 'DCAL', 'DRHO', 'MUDWEIGHT', 'RMIC', 'FORMATION_encoded',
                'WELL_encoded', 'LITHOLOGY']
    for f in features:
        df = df.round({f: 3})  # 保留3位小数点
        df[f] = feature_normalize(df[f])

    df_test = df[df['WELL_encoded'] <= 10]
    df_validation = df[df['WELL_encoded'] in range(11, 21)]
    df_train = df[df['WELL_encoded'] > 20]

    LABEL = "GroupEncoded"
    le = preprocessing.LabelEncoder()
    # 标准化标签，将标签值统一转换成range(标签值个数-1)范围内
    df[LABEL] = le.fit_transform(df["GROUP_encoded"].values.ravel())

    x_train, y_train = create_segments_and_labels(df_train,
                                                  TIME_PERIODS,
                                                  STEP_DISTANCE,
                                                  LABEL,
                                                  features)

    x_valid, y_valid = create_segments_and_labels(df_validation,
                                                  TIME_PERIODS,
                                                  STEP_DISTANCE,
                                                  LABEL,
                                                  features)

    x_test, y_test = create_segments_and_labels(df_test,
                                                TIME_PERIODS,
                                                STEP_DISTANCE,
                                                LABEL,
                                                features)

    return x_train, y_train, x_valid, y_valid, x_test, y_test


# 残差块
def ResBlock(x, filters, kernel_size, dilation_rate):
    r = Conv1D(filters, kernel_size, padding='same', dilation_rate=dilation_rate, activation='relu')(x)  # 第一卷积
    r = Conv1D(filters, kernel_size, padding='same', dilation_rate=dilation_rate)(r)  # 第二卷积
    if x.shape[-1] == filters:
        shortcut = x
    else:
        shortcut = Conv1D(filters, kernel_size, padding='same')(x)  # shortcut（捷径）
    o = add([r, shortcut])
    o = Activation('relu')(o)  # 激活函数
    return o


# 序列模型
def TCN(train_x, train_y, valid_x, valid_y, test_x, test_y):
    inputs = Input(shape=(28, 28))
    x = ResBlock(inputs, filters=32, kernel_size=3, dilation_rate=1)
    x = ResBlock(x, filters=32, kernel_size=3, dilation_rate=2)
    x = ResBlock(x, filters=16, kernel_size=3, dilation_rate=4)
    x = Flatten()(x)
    x = Dense(10, activation='softmax')(x)
    model = Model(input=inputs, output=x)
    # 查看网络结构
    model.summary()
    # 编译模型
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # 训练模型
    model.fit(train_x, train_y, batch_size=500, nb_epoch=30, verbose=2, validation_data=(valid_x, valid_y))
    # 评估模型
    pre = model.evaluate(test_x, test_y, batch_size=500, verbose=2)
    print('test_loss:', pre[0], '- test_acc:', pre[1])


train_x, train_y, valid_x, valid_y, test_x, test_y = read_data(r'D:\workspace\scientificProject\stratigraphic_classfication_correlation\data\force-train.csv')
TCN(train_x, train_y, valid_x, valid_y, test_x, test_y)


keras version  2.6.0

--- Load, inspect and transform data ---

Number of columns in the dataframe: 26
Number of rows in the dataframe: 1170511

First 20 rows of the dataframe:

    Unnamed: 0  Unnamed: 0.1  DEPTH_MD    X_LOC     Y_LOC  Z_LOC  CALI   RSHA  \
0            0             0     494.5 437642.0 6470972.5 -469.5  19.5 -999.0   
1            1             1     494.7 437642.0 6470972.5 -469.7  19.5 -999.0   
2            2             2     494.8 437642.0 6470972.5 -469.8  19.5 -999.0   
3            3             3     495.0 437642.0 6470972.5 -470.0  19.5 -999.0   
4            4             4     495.1 437642.0 6470972.5 -470.1  19.5 -999.0   
5            5             5     495.3 437642.0 6470972.5 -470.3  19.5 -999.0   
6            6             6     495.4 437642.0 6470972.5 -470.4  19.5 -999.0   
7            7             7     495.6 437642.0 6470972.5 -470.6  19.5 -999.0   
8            8             8     495.7 437642.0 6470972.5 -470.7  19.5 -999.0   
9           


--- Reshape the data into segments ---


--- Reshape data to be accepted by Keras ---

x_train shape:  (13645, 160, 23)
13645 training samples
y_train shape:  (13645,)
[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
x_train shape: (13645, 3680)
input_shape: 3680
New y_train shape:  (13645, 15)

--- Create neural network model ---

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 160, 23)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 151, 100)          23100     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 142, 100)          100100    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 47, 100)           0         
_________________________________________________

KeyboardInterrupt: 